### Projeye Dahil Edilmesi Gereken Kütüphaneler

In [ ]:
%pip install ultralytics==8.0.114
%pip install pandas==2.0.2
%pip install opencv-python==4.7.0.72
%pip install numpy==1.24.3
%pip install scipy==1.10.1
%pip install easyocr==1.7.0
%pip install filterpy==1.4.5
%pip install lap==0.4.0
%pip install scikit-image==0.17.2
%pip install pymysql

In [ ]:
from ultralytics import YOLO
import cv2

import util
# from sort.sort import *
from sort import *
from util import get_car, read_license_plate, write_csv


results = {}
mot_tracker = Sort()

modelAdi = "bestM3.pt" #Model adı bu alana eklenecek
videoAdi = "video.mp4" #Video Adı Bu Kısıma Eklenecek

# load models
coco_model = YOLO('yolov8n.pt')
license_plate_detector = YOLO(modelAdi)


# load video
cap = cv2.VideoCapture('./'+videoAdi)

vehicles = [2, 3, 5, 7]

In [ ]:
# read frames
frame_nmr = -1
ret = True
while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        results[frame_nmr] = {}
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

            if car_id != -1:

                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)

                if license_plate_text is not None:

                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}

# write results
write_csv(results, './test.csv')

### Veri tabanı bağlantıları

In [ ]:
# MySQL bağlantısı
import mysql.connector
conn = mysql.connector.connect(
    host="hostIp",
    user="kullanıcıAdı",
    password="parola",
    database="veriTabanı"
)

In [ ]:
# MySQL bağlantısı başarılı mı kontrol et
if conn.is_connected():
    cursor = conn.cursor()

    # Tablo var mı kontrol et
    cursor.execute("SHOW TABLES LIKE 'tablolar'")
    table_exists = cursor.fetchone()

    if not table_exists:
        # Tablo yoksa oluştur
        create_table_query = """
        SET SQL_MODE = "NO_AUTO_VALUE_ON_ZERO";
        START TRANSACTION;
        SET time_zone = "+00:00";

        CREATE TABLE `tablolar` (
          `id` int NOT NULL,
          `tabloNumarasi` int NOT NULL
        ) ENGINE=MyISAM DEFAULT CHARSET=utf8mb4;

        ALTER TABLE `tablolar`
          ADD PRIMARY KEY (`id`);

        ALTER TABLE `tablolar`
          MODIFY `id` int NOT NULL AUTO_INCREMENT, AUTO_INCREMENT=1;
        COMMIT;
        """

        cursor.execute(create_table_query)
        print("Tablo oluşturuldu.")
    else:
        print("Tablo zaten var.")

    # Bağlantıyı kapat
    cursor.close()
    conn.close()
else:
    print("MySQL bağlantısı kurulamadı.")

In [ ]:
# Bir cursor oluşturuyoruz
cursor = conn.cursor()

select_last_row_query = """
SELECT `tabloNumarasi` FROM `tablolar` ORDER BY `id` DESC LIMIT 1;
"""

# Sorguyu çalıştır
cursor.execute(select_last_row_query)

# Sonuçları al
result = cursor.fetchone()

# Eğer sonuç varsa, değeri yazdır
if result:
    print("Son eklenmiş tabloNumarası:", result[0])
    yeniTabloNumarasi = result[0]+1
    yeniTabloAdi = "plakalar{}".format(yeniTabloNumarasi)
    print("Yeni tablo numarası ", yeniTabloAdi)
else:
    print("Tabloda hiç veri yok, tablo ekleniyor.")

In [ ]:
if result is None:
    # Tablo oluşturma SQL sorgusu
    yeniTabloAdi = "plakalar1"
    yeniTabloNumarasi = "1"
    create_table_query = """
    CREATE TABLE `{}` (
    `id` int NOT NULL,
    `aracId` float DEFAULT NULL,
    `aracSiniri` float DEFAULT NULL,
    `plakaSiniri` float DEFAULT NULL,
    `plakaSinirSkoru` float DEFAULT NULL,
    `aracPlakasi` varchar(15) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci DEFAULT NULL,
    `aracPlakaSkoru` float DEFAULT NULL,
    `yorum` varchar(250) COLLATE utf8mb4_general_ci DEFAULT NULL,
    `video` varchar(50) COLLATE utf8mb4_general_ci DEFAULT NULL,
    `model` varchar(50) COLLATE utf8mb4_general_ci DEFAULT NULL,
    `tarih` datetime DEFAULT CURRENT_TIMESTAMP,
    `durum` int DEFAULT '1'
    ) ENGINE=MyISAM DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci;
    """.format(yeniTabloAdi)

    # Tablo için indeks eklenmesi SQL sorgusu
    add_primary_key_query = """
    ALTER TABLE `{}`
    ADD PRIMARY KEY (`id`);
    """.format(yeniTabloAdi)

    # Tablo için AUTO_INCREMENT değeri atanması SQL sorgusu
    set_auto_increment_query = """
    ALTER TABLE `{}`
    MODIFY `id` int NOT NULL AUTO_INCREMENT, AUTO_INCREMENT=105;
    """.format(yeniTabloAdi)

    # Sorguları sırasıyla çalıştırma
    cursor.execute(create_table_query)
    cursor.execute(add_primary_key_query)
    cursor.execute(set_auto_increment_query)

    # Değişiklikleri kaydet
    conn.commit()
    print(yeniTabloAdi, "adıyla yeni bir tablo oluşturuldu.")

    try:
        # Veri ekleme
        cursor.execute(f"INSERT INTO tablolar (tabloNumarasi) VALUES ({yeniTabloNumarasi})")

        # Değişiklikleri kaydet
        conn.commit()

        print(yeniTabloAdi," adlı tablonun numarası tablolar adlı tabloya başarıyla eklendi.")

    except Exception as e:
        print(f"Hata: {e}")


else:
    # Tablo oluşturma SQL sorgusu
    create_table_query = """
    CREATE TABLE `{}` (
    `id` int NOT NULL,
    `aracId` float DEFAULT NULL,
    `aracSiniri` float DEFAULT NULL,
    `plakaSiniri` float DEFAULT NULL,
    `plakaSinirSkoru` float DEFAULT NULL,
    `aracPlakasi` varchar(15) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci DEFAULT NULL,
    `aracPlakaSkoru` float DEFAULT NULL,
    `yorum` varchar(250) COLLATE utf8mb4_general_ci DEFAULT NULL,
    `video` varchar(50) COLLATE utf8mb4_general_ci DEFAULT NULL,
    `model` varchar(50) COLLATE utf8mb4_general_ci DEFAULT NULL,
    `tarih` datetime DEFAULT CURRENT_TIMESTAMP,
    `durum` int DEFAULT '1'
    ) ENGINE=MyISAM DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci;
    """.format(yeniTabloAdi)

    # Tablo için indeks eklenmesi SQL sorgusu
    add_primary_key_query = """
    ALTER TABLE `{}`
    ADD PRIMARY KEY (`id`);
    """.format(yeniTabloAdi)

    # Tablo için AUTO_INCREMENT değeri atanması SQL sorgusu
    set_auto_increment_query = """
    ALTER TABLE `{}`
    MODIFY `id` int NOT NULL AUTO_INCREMENT, AUTO_INCREMENT=105;
    """.format(yeniTabloAdi)

    # Sorguları sırasıyla çalıştırma
    cursor.execute(create_table_query)
    cursor.execute(add_primary_key_query)
    cursor.execute(set_auto_increment_query)

    # Değişiklikleri kaydet
    conn.commit()
    print(yeniTabloAdi, "adıyla yeni bir tablo oluşturuldu")

    try:
        # Veri ekleme
        cursor.execute(f"INSERT INTO tablolar (tabloNumarasi) VALUES ({yeniTabloNumarasi})")

        # Değişiklikleri kaydet
        conn.commit()

        print(yeniTabloAdi," adlı tablonun numarası tablolar adlı tabloya başarıyla eklendi.")

    except Exception as e:
        print(f"Hata: {e}")

In [ ]:
import pandas as pd
# CSV dosyasını oku
csv_file_path = "test.csv"
df = pd.read_csv(csv_file_path)

# Verileri MySQL tablosuna ekle
for index, row in df.iterrows():
    insert_query = f"INSERT INTO {yeniTabloAdi} (aracId, aracSiniri, plakaSiniri, plakaSinirSkoru, aracPlakasi, aracPlakaSkoru, video, model) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    values = (row['car_id'], row['car_bbox'], row['license_plate_bbox'], row['license_plate_bbox_score'], row['license_number'], row['license_number_score'], videoAdi, modelAdi)
    cursor.execute(insert_query, values)

# Değişiklikleri kaydet
conn.commit()

# Bağlantıyı kapat
conn.close()

print("Veriler başarıyla ",yeniTabloAdi," tablosuna aktarıldı.")

# Performans Değerlendirmeleri

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
from io import BytesIO

# MySQL bağlantısı
conn = mysql.connector.connect(
    host="hostIp",
    user="kullanıcıAdı",
    password="parola",
    database="veriTabanı"
)

tabloAdi = yeniTabloAdi+yeniTabloNumarasi

# SQL sorgusu
sql_query = "SELECT * FROM "+tabloAdi

# Veriyi al
df = pd.read_sql_query(sql_query, conn)

# Bağlantıyı kapat
conn.close()

# "0" olmayan aracPlakasi'na sahip satırları filtrele
filtered_df = df[df['aracPlakasi'] != '0']

# aracId'ye göre grupla ve en yüksek aracPlakaSkoru'na sahip satırı bul
max_performance_rows = filtered_df.loc[filtered_df.groupby('aracId')['aracPlakaSkoru'].idxmax()]

# aracId'ye göre grupla ve en yaygın aracPlakaSkoru'nu bul
most_common_performance = filtered_df.groupby('aracId')['aracPlakaSkoru'].agg(lambda x: x.mode().iloc[0])

# Seaborn kullanarak veriyi çiz
plt.figure(figsize=(10, 6))
sns.barplot(x=most_common_performance.index, y=most_common_performance.values)
plt.xlabel('Araçlar')
plt.ylabel('En Yaygın Araç Plaka Skoru')
plt.title('Her Arac İçin En Yaygın Araç Plaka Skoru')
plt.savefig(tabloAdi+'_.png')

# Grafiği BytesIO nesnesine kaydet
image_stream = BytesIO()
plt.savefig(image_stream, format='png')
image_stream.seek(0)

# Şimdi, image_stream PNG resim verisini içeriyor
# İhtiyacınıza göre bir dosyaya kaydedebilir veya kullanabilirsiniz

# Grafiği göster (isteğe bağlı)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
from io import BytesIO

# MySQL bağlantısı
conn = mysql.connector.connect(
    host="hostIp",
    user="kullanıcıAdı",
    password="parola",
    database="veriTabanı"
)

tabloAdi = yeniTabloAdi+yeniTabloNumarasi

# SQL sorgusu
sql_query = "SELECT * FROM "+tabloAdi

# Veriyi al
df = pd.read_sql_query(sql_query, conn)

# Bağlantıyı kapat
conn.close()

# "0" olmayan aracPlakasi'na sahip satırları filtrele
filtered_df = df[df['aracPlakasi'] != '0']

# aracId ve aracPlakaSkoru'na göre grupla ve en yüksek aracPlakaSkoru'na sahip satırı bul
max_performance_rows = filtered_df.loc[filtered_df.groupby(['aracId', 'aracPlakaSkoru'])['aracPlakaSkoru'].idxmax()]

# Seaborn kullanarak veriyi çiz
plt.figure(figsize=(10, 6))
sns.barplot(x=max_performance_rows['aracId'], y=max_performance_rows['aracPlakaSkoru'])
plt.xlabel('Araçlar')
plt.ylabel('En Yüksek Araç Plaka Skoru')
plt.title('Her Araç İçin En Yüksek Araç Plaka Skoru')
plt.savefig(tabloAdi+".png")

# Grafiği BytesIO nesnesine kaydet
image_stream = BytesIO()
plt.savefig(image_stream, format='png')
image_stream.seek(0)

# Şimdi, image_stream PNG resim verisini içeriyor
# İhtiyacınıza göre bir dosyaya kaydedebilir veya kullanabilirsiniz

# Grafiği göster (isteğe bağlı)
plt.show()
